In [1]:
import pandas as pd
import numpy as np
import QuantLib as ql
from pathlib import Path

DATA_DIR = Path("../data")

DATA_Q4 = pd.read_excel(DATA_DIR / "FI.xlsm", sheet_name="Q4", header=2, usecols="E:F")
DATA_Q4.columns = ["Days", "DF"]
DATA_Q4.dropna(inplace=True)

DATA_HISTORICAL_EURIBOR = pd.read_excel(DATA_DIR / "FI.xlsm", sheet_name="Historical Euribor", header=4, usecols="A:B")
DATA_HISTORICAL_EURIBOR.columns = ["Date", "3M EURIBOR"]
DATA_HISTORICAL_EURIBOR["Date"] = pd.to_datetime(DATA_HISTORICAL_EURIBOR["Date"])
DATA_HISTORICAL_EURIBOR.set_index("Date", inplace=True)
EURIBOR_SERIES = DATA_HISTORICAL_EURIBOR["3M EURIBOR"].astype(float) / 100

# QuantLib setup - check documentation on their website - https://quantlib-python-docs.readthedocs.io/en/latest/
VALUATION_DATE = ql.Date(26, 11, 2024)
ql.Settings.instance().evaluationDate = VALUATION_DATE
QL_DATES = [VALUATION_DATE + int(d) for d in DATA_Q4["Days"]]
ql_dfs = DATA_Q4["DF"].tolist()
curve = ql.DiscountCurve(QL_DATES, ql_dfs, ql.Actual365Fixed(), ql.TARGET())
curve.enableExtrapolation()
CURVE_HANDLE = ql.YieldTermStructureHandle(curve)

# Bond parameters
NOTIONAL = 1000
issue_date = ql.Date(29, 7, 2022)
maturity_date = ql.Date(29, 7, 2027)
schedule = ql.Schedule(issue_date, maturity_date, ql.Period("3M"), ql.TARGET(),
                       ql.ModifiedFollowing, ql.ModifiedFollowing,
                       ql.DateGeneration.Forward, False)
floor_rate = 0.016
cap_rate = 0.037
day_count = ql.Actual360()
calendar = ql.TARGET()

# Helpers
def ql_to_datetime(qdate):
    return pd.Timestamp(qdate.year(), qdate.month(), qdate.dayOfMonth())

def get_euribor_fixing(reset_date, start, end):
    try:
        return EURIBOR_SERIES.loc[ql_to_datetime(reset_date)]
    except KeyError:
        return CURVE_HANDLE.forwardRate(start, end, day_count, ql.Simple).rate()

# Cash flow build
rows = []
for i in range(1, len(schedule)):
    start = schedule[i-1]
    end = schedule[i]
    reset = calendar.advance(start, -2, ql.Days)
    accrual = day_count.yearFraction(start, end)

    ref_rate = get_euribor_fixing(reset, start, end)
    capped_rate = min(max(ref_rate, floor_rate), cap_rate)

    df = 1.0 if end < VALUATION_DATE else CURVE_HANDLE.discount(end)
    coupon = NOTIONAL * ref_rate * accrual
    capped_coupon = NOTIONAL * capped_rate * accrual

    rows.append([
        ql_to_datetime(start), ql_to_datetime(end), ql_to_datetime(reset),
        round(accrual, 6), round(ref_rate * 100, 4), round(capped_rate * 100, 4),
        round(coupon, 2), round(capped_coupon, 2), round(df, 6),
        round(df * coupon, 2), round(df * capped_coupon, 2)
    ])

columns = ["Start", "End", "Reset", "Accrual", "Euribor Rate (%)", "Cap/Floor Rate (%)",
           "Coupon (€)", "Cap/Floor Coupon (€)", "DF", "PV (€)", "PV Cap/Floor (€)"]
cf_df = pd.DataFrame(rows, columns=columns)
cf_df.tail(11)

,Start,End,Reset,Accrual,Euribor Rate (%),Cap/Floor Rate (%),Coupon (€),Cap/Floor Coupon (€),DF,PV (€),PV Cap/Floor (€)
9,2024-10-29,2025-01-29,2024-10-25,0.255556,3.0590,3.0590,7.82,7.82,0.994724,7.78,7.78
10,2025-01-29,2025-04-29,2025-01-27,0.250000,2.6410,2.6410,6.60,6.60,0.988269,6.53,6.53
11,2025-04-29,2025-07-29,2025-04-25,0.252778,2.4804,2.4804,6.27,6.27,0.982112,6.16,6.16
12,2025-07-29,2025-10-29,2025-07-25,0.255556,1.7927,1.7927,4.58,4.58,0.977633,4.48,4.48
13,2025-10-29,2026-01-29,2025-10-27,0.255556,1.8079,1.8079,4.62,4.62,0.973137,4.50,4.50
14,2026-01-29,2026-04-29,2026-01-27,0.250000,1.9774,1.9774,4.94,4.94,0.968349,4.79,4.79
15,2026-04-29,2026-07-29,2026-04-27,0.252778,1.9775,1.9775,5.00,5.00,0.963533,4.82,4.82
16,2026-07-29,2026-10-29,2026-07-27,0.255556,1.9775,1.9775,5.05,5.05,0.958688,4.84,4.84
17,2026-10-29,2027-01-29,2026-10-27,0.255556,2.0750,2.0750,5.30,5.30,0.953631,5.06,5.06
18,2027-01-29,2027-04-29,2027-01-27,0.250000,2.1220,2.1220,5.31,5.31,0.948599,5.03,5.03


In [2]:
import matplotlib.pyplot as plt

def apply_shift(spot_curve,shift_type,shift):
    t1 = DATA_Q4['Days'].min()
    tm = DATA_Q4['Days'].median()
    tn = DATA_Q4['Days'].max()
    b = (-shift-shift) / (tn-t1)

    if shift_type == 'parallel':
        return [x + shift for x in spot_curve]
    
    elif shift_type == 'slope':
        shifts = np.arange(shift, shift+len(spot_curve)*b, b)
        return [x + y for x, y in zip(spot_curve, shifts)]
        
    elif shift_type == 'convexity':
        X = np.array([
            [1, 0, 0],
            [1, tm, tm**2],
            [1, tn, tn**2]
        ])
        Y = np.array([shift, -shift, shift]).reshape(3,1)
        beta = np.linalg.inv(X) @ Y
        a,b,c = beta.flatten()
        
        shifts = [a + b*x + c*x**2 for x in DATA_Q4['Days']]
        return [x + y for x, y in zip(spot_curve, shifts)]

def compute_bond_clean_price(dfs):
    ql_dates = [VALUATION_DATE + int(d) for d in DATA_Q4["Days"]]
    curve = ql.DiscountCurve(ql_dates, dfs.tolist(), ql.Actual365Fixed(), ql.TARGET())
    curve.enableExtrapolation()
    handle = ql.YieldTermStructureHandle(curve)

    pv = 0.0
    for i, row in future_cf.iterrows():
        qldate = ql.Date(row["End"].day, row["End"].month, row["End"].year)
        pv += row["Cap/Floor Coupon (€)"] * handle.discount(qldate)
        if i == len(future_cf) - 1:
            pv += NOTIONAL * handle.discount(qldate)
    return pv

def compute_swap_clean_price(dfs, swap_type='payer'):
    swap_rate = 0.0202 # Quote from EUR 3Y AB3E IRS

    curve = ql.DiscountCurve(QL_DATES, dfs.tolist(), ql.Actual365Fixed(), ql.TARGET())
    curve.enableExtrapolation()
    handle = ql.YieldTermStructureHandle(curve)
    
    # Assume valuation and maturity dates of the swap are the same as the bond
    valuation_date = ql.Date(26, 11, 2024) 
    maturity_date = ql.Date(26, 11, 2027) 

    fixed_tenor = ql.Period('1Y')
    floating_tenor = ql.Period('6M')

    fixed_schedule = ql.Schedule(valuation_date,maturity_date,fixed_tenor,ql.TARGET(),
                                 ql.ModifiedFollowing,ql.ModifiedFollowing,
                                 ql.DateGeneration.Backward,False)
    
    floating_schedule = ql.Schedule(valuation_date,maturity_date,floating_tenor,ql.TARGET(),
                                 ql.ModifiedFollowing,ql.ModifiedFollowing,
                                 ql.DateGeneration.Backward,False)
    
    
    # Price the floating leg
    pv_floating_leg = []
    for i in range(1, len(floating_schedule)):
        start, end = floating_schedule[i-1], floating_schedule[i]
        fwd_rate = handle.forwardRate(start, end, ql.Actual360(), ql.Simple).rate()
        accrual_factor = ql.Actual360().yearFraction(start, end)
        
        pv_floating_leg.append(NOTIONAL * fwd_rate * handle.discount(end) * accrual_factor)

    # Price the fixed leg
    pv_fixed_leg = []
    for i in range(1, len(fixed_schedule)):
        start, end = fixed_schedule[i-1], fixed_schedule[i]
        
        accrual_factor = ql.Thirty360(ql.Thirty360.EurobondBasis).yearFraction(start, end)
        
        pv_fixed_leg.append(NOTIONAL * swap_rate * handle.discount(end) * accrual_factor)
    
    if swap_type == 'payer':
        return np.sum(pv_floating_leg) - np.sum(pv_fixed_leg)
    else:
        return np.sum(pv_fixed_leg) - np.sum(pv_floating_leg)

DF = DATA_Q4['DF'].tolist()
SPOT_RATE = (-np.log(DATA_Q4['DF']) / (DATA_Q4['Days']/365)).fillna(1)

SLOPE_SHIFT = 0.0001
PARALLEL_SHIFT = 0.0001
CURVE_SHIFT = 0.0001

future_cf = cf_df[cf_df["End"] > pd.Timestamp("2024-11-26")].copy().sort_values("End").reset_index(drop=True)
shifted_scenarios = {
    "Parallel Up": apply_shift(SPOT_RATE, 'parallel', PARALLEL_SHIFT),
    "Parallel Down": apply_shift(SPOT_RATE, 'parallel', -PARALLEL_SHIFT),
    "Slope Up": apply_shift(SPOT_RATE, 'slope', SLOPE_SHIFT),
    "Slope Down": apply_shift(SPOT_RATE, 'slope', -SLOPE_SHIFT),
    "Curvature Up": apply_shift(SPOT_RATE, 'convexity', CURVE_SHIFT),
    "Curvature Down": apply_shift(SPOT_RATE, 'convexity', -CURVE_SHIFT)
}

accrued_interest = 2.38
bond_gross_value = compute_bond_clean_price(DATA_Q4['DF']) + accrued_interest
swap_gross_value = compute_swap_clean_price(DATA_Q4['DF']) + accrued_interest

bond_shift_results = {}
swap_shift_results = {}
for name, shifted_spot_rates in shifted_scenarios.items():
    shifted_dfs = np.array(np.exp(-np.array(shifted_spot_rates) * (DATA_Q4['Days'] / 365), dtype=float))
    swap_shift_results[name] = compute_swap_clean_price(shifted_dfs) + accrued_interest
    bond_shift_results[name] = compute_bond_clean_price(shifted_dfs) + accrued_interest

table_data = {
    "Shifts": ["Parallel", "Slope", "Convexity"],
    "Up Bond (€)": [bond_shift_results["Parallel Up"], bond_shift_results["Slope Up"], bond_shift_results["Curvature Up"]],
    "Down Bond (€)": [bond_shift_results["Parallel Down"], bond_shift_results["Slope Down"], bond_shift_results["Curvature Down"]],
    "Up Swap (€)": [swap_shift_results["Parallel Up"], swap_shift_results["Slope Up"], swap_shift_results["Curvature Up"]],
    "Down Swap (€)": [swap_shift_results["Parallel Down"], swap_shift_results["Slope Down"], swap_shift_results["Curvature Down"]],
}

def calc_dv01(up,down):
    return -(up - down) / 2

# Compute DV01
table_data["DV01 Bond (€)"] = [
    calc_dv01(bond_shift_results["Parallel Up"], bond_shift_results["Parallel Down"]),
    calc_dv01(bond_shift_results["Slope Up"], bond_shift_results["Slope Down"]),
    calc_dv01(bond_shift_results["Curvature Up"], bond_shift_results["Curvature Down"])
]
table_data["DV01 Swap (€)"] = [
    calc_dv01(swap_shift_results["Parallel Up"], swap_shift_results["Parallel Down"]),
    calc_dv01(swap_shift_results["Slope Up"], swap_shift_results["Slope Down"]),
    calc_dv01(swap_shift_results["Curvature Up"], swap_shift_results["Curvature Down"])
]
table_data["Hedge Ratio"] = [
    (-table_data["DV01 Bond (€)"][i] / table_data["DV01 Swap (€)"][i]) if table_data["DV01 Bond (€)"][i] != 0 else 0
    for i in range(len(table_data["Shifts"]))
]

print(f'Initial Swap Gross Value (€): {swap_gross_value:.4f}')
print(f'Initial Gross Risk-free Value (€): {bond_gross_value:.4f}')
display(pd.DataFrame(table_data))

Initial Swap Gross Value (€): 7.4880
Initial Gross Risk-free Value (€): 1004.9418


,Shifts,Up Bond (€),Down Bond (€),Up Swap (€),Down Swap (€),DV01 Bond (€),DV01 Swap (€),Hedge Ratio
0,Parallel,1004.681906,1005.201695,7.780503,7.195326,0.259895,-0.292589,0.888258
1,Slope,1004.704806,1005.178784,7.751537,7.224309,0.236989,-0.263614,0.898998
2,Convexity,1004.769453,1005.114110,7.670396,7.305487,0.172329,-0.182454,0.944502


In [3]:
# Hedge implementation

# Long bond + long swap

shift_levels = [0.0002, 0.0005, 0.001]  # 2bps, 5bps, 10bps
shift_types = ['parallel', 'slope', 'convexity']
HEDGE_RATIO = table_data["Hedge Ratio"][0]

results = []

for shift in shift_levels:
    shifted_scenarios = {
        f"{stype.title()} Up": apply_shift(SPOT_RATE, stype, shift)
        for stype in shift_types
    }
    shifted_scenarios.update({
        f"{stype.title()} Down": apply_shift(SPOT_RATE, stype, -shift)
        for stype in shift_types
    })

    portfolio_shift_results = {}

    for scenario_name, shifted_spot_rates in shifted_scenarios.items():
        shifted_dfs = np.array(np.exp(-np.array(shifted_spot_rates) * (DATA_Q4['Days'] / 365), dtype=float))
        
        bond_price = compute_bond_clean_price(shifted_dfs) + accrued_interest
        swap_price = HEDGE_RATIO * compute_swap_clean_price(shifted_dfs) + accrued_interest
        
        portfolio_shift_results[scenario_name] = bond_price + swap_price

    # Calculate DV01 for each shift type
    for stype in shift_types:
        up_value = portfolio_shift_results[f"{stype.title()} Up"]
        down_value = portfolio_shift_results[f"{stype.title()} Down"]
        
        dv01 = calc_dv01(up_value, down_value)
        
        results.append({
            "Shift Type": stype.title(),
            "Shift (bps)": int(shift*10000),
            "DV01 Portfolio (€)": dv01
        })

# Display result as DataFrame
dv01_df = pd.DataFrame(results)
dv01_df = dv01_df.pivot(index='Shift Type', columns='Shift (bps)', values='DV01 Portfolio (€)')
display(dv01_df.round(6))

Shift (bps),2,5,10
Shift Type,,,
Convexity,0.020524,0.051310,0.102620
Parallel,-0.000000,-0.000000,-0.000001
Slope,0.005662,0.014156,0.028311
